In [1]:
#pip install surprise

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor,KNNBasic,KNNWithMeans,KNNWithZScore,KNNBaseline,SVD,BaselineOnly,SVDpp,NMF,SlopeOne,CoClustering
from surprise.accuracy import rmse
from surprise import accuracy

In [3]:
users = pd.read_csv('users.csv', sep='\";\"', names=['User-ID', 'Location', 'Age'], encoding='latin-1', skiprows=1)
books = pd.read_csv('books.csv', sep='\";\"', names=['ISBN', 'Book-Title' ,'Book-Author','Year-Of-Publication', 'Publisher', 'Image-Url-S', 'Image-Url-M', 'Image-Url-L'], encoding='latin-1', skiprows=1)
ratings = pd.read_csv('ratings.csv', sep='\";\"', names=['User-ID', 'ISBN', 'Book-Rating'], encoding='latin-1', skiprows=1)

<ipython-input-3-1dc5693745f4>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv('users.csv', sep='\";\"', names=['User-ID', 'Location', 'Age'], encoding='latin-1', skiprows=1)
<ipython-input-3-1dc5693745f4>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  books = pd.read_csv('books.csv', sep='\";\"', names=['ISBN', 'Book-Title' ,'Book-Author','Year-Of-Publication', 'Publisher', 'Image-Url-S', 'Image-Url-M', 'Image-Url-L'], encoding='latin-1', skiprows=1)
<ipython-input-3-1dc5693745f4>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex 

In [4]:
print(users)

        User-ID                                  Location   Age
0            "1                  nyc, new york, usa";NULL  None
1            "2                 stockton, california, usa   18"
2            "3     moscow, yukon territory, russia";NULL  None
3            "4                 porto, v.n.gaia, portugal   17"
4            "5  farnborough, hants, united kingdom";NULL  None
...         ...                                       ...   ...
278854  "278854               portland, oregon, usa";NULL  None
278855  "278855        tacoma, washington, united kingdom   50"
278856  "278856           brampton, ontario, canada";NULL  None
278857  "278857           knoxville, tennessee, usa";NULL  None
278858  "278858                dublin, n/a, ireland";NULL  None

[278859 rows x 3 columns]


In [5]:
print(books)

               ISBN                                         Book-Title  \
0       "0195153448                                Classical Mythology   
1       "0002005018                                       Clara Callan   
2       "0060973129                               Decision in Normandy   
3       "0374157065  Flu: The Story of the Great Influenza Pandemic...   
4       "0393045218                             The Mummies of Urumchi   
...             ...                                                ...   
271374  "0440400988                         There's a Bat in Bunk Five   
271375  "0525447644                            From One to One Hundred   
271376  "006008667X  Lily Dale : The True Story of the Town that Ta...   
271377  "0192126040                        Republic (World's Classics)   
271378  "0767409752  A Guided Tour of Rene Descartes' Meditations o...   

                 Book-Author  Year-Of-Publication  \
0         Mark P. O. Morford                 2002   
1    

In [6]:
print(ratings)

         User-ID         ISBN Book-Rating
0        "276725   034545104X          0"
1        "276726   0155061224          5"
2        "276727   0446520802          0"
3        "276729   052165615X          3"
4        "276729   0521795028          6"
...          ...          ...         ...
1149775  "276704   1563526298          9"
1149776  "276706   0679447156          0"
1149777  "276709   0515107662         10"
1149778  "276721   0590442449         10"
1149779  "276723  05162443314          8"

[1149780 rows x 3 columns]


In [7]:
users.describe()

,User-ID,Location,Age
count,278859,278858,168096
unique,278859,70328,165
top,"""130823","london, england, united kingdom","24"""
freq,1,1629,5687


In [8]:
books.describe()

,Year-Of-Publication
count,271379.000000
mean,1959.756050
std,258.011363
min,0.000000
25%,1989.000000
50%,1995.000000
75%,2000.000000
max,2050.000000


In [9]:
ratings.describe()

,User-ID,ISBN,Book-Rating
count,1149780,1149780,1149780
unique,105283,340556,11
top,"""11676",0971880107,"0"""
freq,13602,2502,716109


In [10]:
users['User-ID'] = users['User-ID'].str.replace("\"","")
users['Location'] = users['Location'].str.replace("\";NULL","")
users['Age'] = users['Age'].fillna("0")
users['Age'] = users['Age'].str.replace("\"","")
books['ISBN'] = books['ISBN'].str.replace("\"","")
books['Book-Title'] = books['Book-Title'].str.replace("\"","")
ratings['User-ID'] = ratings['User-ID'].str.replace("\"","")
ratings['Book-Rating'] = ratings['Book-Rating'].str.replace("\"","").astype(int)

In [11]:
quality_ratings = ratings[ratings['Book-Rating']!=0]
quality_book = quality_ratings['ISBN'].value_counts().rename_axis('ISBN').reset_index(name = 'Count')
quality_book = quality_book[quality_book['Count']>5]['ISBN'].to_list()
quality_ratings = quality_ratings[quality_ratings['ISBN'].isin(quality_book)]
quality_ratings

,User-ID,ISBN,Book-Rating
8,276744,038550120X,7
16,276747,0060517794,9
19,276747,0671537458,9
20,276747,0679776818,8
28,276754,0684867621,8
...,...,...,...
1149743,276688,0836218655,10
1149744,276688,0836236688,10
1149761,276704,0345386108,6
1149771,276704,0743211383,7


In [12]:
quality_user = quality_ratings['User-ID'].value_counts().rename_axis('User-ID').reset_index(name = 'Count')
quality_user = quality_user[quality_user['Count']>5]['User-ID'].to_list()
quality_ratings = quality_ratings[quality_ratings['User-ID'].isin(quality_user)]
quality_ratings

,User-ID,ISBN,Book-Rating
174,276847,3257200552,5
177,276847,3404148576,8
179,276847,3423071516,10
182,276847,3426029553,8
186,276847,3442413508,10
...,...,...,...
1149715,276688,0553575104,6
1149738,276688,0688156134,8
1149739,276688,0743202694,10
1149743,276688,0836218655,10


In [13]:
mean_rating_user = quality_ratings.groupby('User-ID')['Book-Rating'].mean().reset_index(name='Mean-Rating-User')
mean_data = pd.merge(quality_ratings, mean_rating_user, on='User-ID')
mean_data['Diff'] = mean_data['Book-Rating'] - mean_data['Mean-Rating-User']
mean_data['Square'] = (mean_data['Diff'])**2
norm_data = mean_data.groupby('User-ID')['Square'].sum().reset_index(name='Mean-Square')
norm_data['Root-Mean-Square'] = np.sqrt(norm_data['Mean-Square'])
mean_data = pd.merge(norm_data, mean_data, on='User-ID')
mean_data['Norm-Rating'] = mean_data['Diff']/(mean_data['Root-Mean-Square'])  
mean_data['Norm-Rating'] = mean_data['Norm-Rating'].fillna(0)
max_rating = mean_data.sort_values('Norm-Rating')['Norm-Rating'].to_list()[-1]
min_rating = mean_data.sort_values('Norm-Rating')['Norm-Rating'].to_list()[0]
mean_data['Norm-Rating'] = 5*(mean_data['Norm-Rating'] - min_rating)/(max_rating-min_rating)
mean_data['Norm-Rating'] = np.ceil(mean_data['Norm-Rating']).astype(int)
norm_ratings = mean_data[['User-ID','ISBN','Norm-Rating']]
mean_data.sort_values('Norm-Rating')

,User-ID,Mean-Square,Root-Mean-Square,ISBN,Book-Rating,Mean-Rating-User,Diff,Square,Norm-Rating
14735,124989,0.956522,0.978019,0061092096,9,9.956522,-0.956522,0.914934,0
54488,213760,56.769231,7.534536,0671693816,3,7.692308,-4.692308,22.017751,1
20494,136011,27.714286,5.264436,055328942X,3,7.428571,-4.428571,19.612245,1
47555,198114,53.875000,7.339959,037541200X,1,6.625000,-5.625000,31.640625,1
31142,160541,102.000000,10.099505,0749324791,2,8.000000,-6.000000,36.000000,1
...,...,...,...,...,...,...,...,...,...
98100,5815,21.333333,4.618802,0316693251,10,7.333333,2.666667,7.111111,5
98082,58067,12.000000,3.464102,0062512668,10,8.000000,2.000000,4.000000,5
76394,259430,29.666667,5.446712,0679781587,10,5.833333,4.166667,17.361111,5
76499,259734,15.428571,3.927922,0441003257,10,7.714286,2.285714,5.224490,5


In [14]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(norm_ratings[['User-ID', 'ISBN', 'Norm-Rating']], reader)

# Machine Learning and Matrix Factorization Models

In [15]:
benchmark = []
for algorithm in [SVD(), 
                  SVDpp(), 
                  SlopeOne(), 
                  NMF(), 
                  NormalPredictor(), 
                  KNNBaseline(), 
                  KNNBasic(), 
                  KNNWithMeans(),
                  KNNWithZScore(), 
                  BaselineOnly(),
                  CoClustering()]:
    
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


0.62 of RMSE says that predicted rating may have an error of 0.62

In [16]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
surprise_results

,test_rmse,fit_time,test_time
Algorithm,,,
BaselineOnly,0.623739,0.107471,0.165700
SVD,0.637767,3.883340,0.166011
SVDpp,0.642748,62.086101,2.691105
KNNBasic,0.651744,1.073407,0.811284
KNNBaseline,0.657618,1.219115,0.972412
KNNWithMeans,0.719125,1.137190,0.897885
KNNWithZScore,0.731071,1.264972,0.874334
SlopeOne,0.743856,5.022980,1.928043
NMF,0.752710,4.220082,0.144135


# Machine Learning based Model

BaselineOnly

Algorithm predicting the baseline estimate for given user and item.

𝑏𝑢𝑖=μ+𝑏𝑢+𝑏𝑖
 
If user u is unknown, then the bias bu is assumed to be zero. The same applies for item i with bi.

using SGD: Stocahstic Gradient Descent to minimize the loss with regularization parameter 0.5

In [17]:
train_set, test_set = train_test_split(data, test_size=0.25)
algo = BaselineOnly(bsl_options={'method': 'sgd','learning_rate': .00005, 'n_epochs':30, 'reg':0.5})
fit = algo.fit(train_set)
pred = fit.test(test_set)
accuracy.rmse(pred)

Estimating biases using sgd...
RMSE: 0.6158


0.6158366659799439

# Matrix Factorization Method

SVD

The famous SVD algorithm, as popularized by Simon Funk during the Netflix Prize. When baselines are not used, This is equivalent to Probabilistic Matrix.

The prediction rui is set as:
𝑟𝑢𝑖=μ+𝑏𝑢+𝑏𝑖+𝑞𝑇𝑖𝑝𝑢

To estimate all the unknown, we minimize the following regularized squared error:

∑𝑟𝑢𝑖∈𝑅𝑡𝑟𝑎𝑖𝑛(𝑟𝑢𝑖−𝑟𝑢𝑖)2+λ(𝑏2𝑖+𝑏2𝑢+||𝑞𝑖||2+||𝑝𝑢||2)

In [18]:
# SVD 

algo = SVD(reg_bi = 0.5, lr_bi=0.005)
fit = algo.fit(train_set)
pred = fit.test(test_set)
accuracy.rmse(pred)

RMSE: 0.6358


0.635775226783601

In [19]:
recommend = algo.trainset
users_norm = list(set(norm_ratings['User-ID'].to_list()))
books_norm = list(set(norm_ratings['ISBN'].to_list()))
norm_ratings['User-ID'].unique()

array(['100009', '100053', '100066', ..., '99738', '99955', '99996'],
      dtype=object)

In [21]:
pred_users = [user for user in users_norm if recommend.knows_user(recommend.to_inner_uid(user))]
pred_books = []
for book in books_norm:
    try:
        if recommend.knows_item(recommend.to_inner_iid(book)):
            pred_books.append(book)
    except:
        pass

ValueError: User 152036 is not part of the trainset.

In [ ]:
pred_users[:5]

# Recommendation Evaluation

In [ ]:
def recommend_books(user_id, count):
    result=[]
    for b in pred_books:
        result.append([b,algo.predict(user_id,b,r_ui=4).est])
    recom = pd.DataFrame(result, columns=['ISBN','Rating'])
    merge = pd.merge(recom,books, on='ISBN' )
    return merge.sort_values('Rating', ascending=False).head(count)

In [ ]:
recommendation = recommend_books('36938', 5)

# Scoring

Scoring
After candidate generation, another model scores and ranks the generated candidates to select the set of items to display. The recommendation system may have multiple candidate generators that use different sources, such as the following:

User features that account for personalization.
geographic information into account.
Popular or trending items.
Here scoring is done based on published year

In [ ]:
scoring = recommendation.sort_values('Year-Of-Publication')
view = "".join(["<span><img src='"+a+"'></span>" for a in scoring['Image-Url-M'].to_list()])
scoring[['Book-Title']]

# Transforming data into the matrix
Next, we transform the dataset into a matrix where each row represents the user and column represents the item.
The n_users defines the number of users whereas n_items defines the number of items in the dataset. The loop iterates through each row of the dataframe and extracts ratings from it to form the user-item rating matrix.

In [ ]:
n_users = dataset.user_id.unique().shape[0]
n_items = dataset.item_id.unique().shape[0]
n_items = dataset['item_id'].max()

print(n_users, n_items)